In [63]:
# importing required libraries in python 
import pandas as pd
import numpy as np
from datetime import datetime
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [64]:
# Reading the data from csv file and parsing the date to making it as index

df_base=pd.read_csv('data/processed/Recovered_processed.csv',
                       parse_dates=[0])  

df_base.sort_values('date',ascending=True).tail()

,date,India,US,Russia,Turkey,Brazil
550,2021-07-25,30579106,0,5424361,5421148,17702038
551,2021-07-26,30621469,0,5440451,5426733,17704701
552,2021-07-27,30663147,0,5459932,5433841,17715481
553,2021-07-28,30701612,0,5480150,5438038,17742532
554,2021-07-29,30743972,0,5500409,5443501,17771228


In [65]:
# function to create a graph
def my_graph(x_in,df_in,y='log',slider=False):
    """          
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
    
        
        Returns:
        ----------
        
    """
    fig = go.Figure()

    for each in df_in.columns:
        fig.add_trace(go.Scatter(
                        x=x_in,
                        y=df_in[each],
                        name=each,
                        opacity=0.8))       
        
    
    fig.update_layout(autosize=True,title="India's Covid-19 Recovered cases forecast",
            font=dict(
            size=18,
            color="#7f7f7f"
            )
        )
    fig.update_yaxes(type=y),
    fig.update_xaxes(tickfont=dict(size=14,color="#7f7f7f"))
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show()

In [66]:
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=False)

In [67]:
#train set
no_days=len(df_base['India'])
X=np.arange(no_days-8).reshape(-1, 1) # -8 : to match the number of records with y
y=np.array(df_base['India'][8:]) # [8:] removing initial 0 cases to handle devide by zero for log

In [68]:
reg.fit(X,y) #simple regression 

LinearRegression(fit_intercept=False)

In [69]:
# test set n Forecasting
X_hat=np.arange(no_days).reshape(-1, 1)
Y_hat=reg.predict(X_hat)

In [70]:
data_LRF=df_base[['date','India']].copy()

In [71]:
data_LRF['Forecast']=Y_hat

In [72]:
data_LRF.head()

,date,India,Forecast
0,2020-01-22,0,0.000000
1,2020-01-23,0,36812.673430
2,2020-01-24,0,73625.346861
3,2020-01-25,0,110438.020291
4,2020-01-26,0,147250.693721


In [73]:
my_graph(data_LRF.date,
           data_LRF.iloc[:,1:],
           y='linear',
           slider=True)

In [74]:
#train set-- converting cases into logarithmic values
no_days=len(df_base['India'])
X1=np.arange(no_days-51).reshape(-1, 1) # -8 : to match the number of records with y
y1=np.log(np.array(df_base['India'][51:])) # [8:] removing initial 0 cases to handle devide by zero for log

In [75]:
reg.fit(X1,y1)

LinearRegression(fit_intercept=False)

In [76]:
# test set n Forecasting
X_hat1=np.arange(no_days).reshape(-1, 1)
Y_hat1=reg.predict(X_hat)

In [77]:
data_LRF1=df_base[['date','India']].copy()

In [78]:
data_LRF1['Forecast']=np.exp(Y_hat1)

In [79]:
data_LRF1.tail()

,date,India,Forecast
550,2021-07-25,30579106,2.373724e+11
551,2021-07-26,30621469,2.489504e+11
552,2021-07-27,30663147,2.610931e+11
553,2021-07-28,30701612,2.738281e+11
554,2021-07-29,30743972,2.871842e+11


In [80]:
my_graph(data_LRF1.date,
           data_LRF1.iloc[:,1:],
           y='log',
           slider=True)